In [1]:
import copy
import os

# ignore bnb warnings
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

if "RANK_TABLE_FILE" in os.environ:
    del os.environ["RANK_TABLE_FILE"]

In [2]:
import mindspore as ms
from mindspore.common.api import _no_grad
from mindnlp.core import nn, ops
import mindnlp.core.nn.functional as F
from mindnlp import peft

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [3]:
ms.manual_seed(0)

In [4]:
X = ops.rand((1000, 20))
y = (X.sum(1) > 10).long()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [5]:
n_train = 800
batch_size = 64

In [6]:
X_train = ms.Tensor(X[:n_train])
y_train = ms.Tensor(y[:n_train])
train_dataset = ms.dataset.NumpySlicesDataset((X_train.asnumpy(), y_train.asnumpy()), column_names=["input_ids", "labels"], shuffle=True)
train_dataset = train_dataset.batch(batch_size)

X_eval = ms.Tensor(X[n_train:])
y_eval = ms.Tensor(y[n_train:])
eval_dataset = ms.dataset.NumpySlicesDataset((X_train.asnumpy(), y_train.asnumpy()), column_names=["input_ids", "labels"], shuffle=False)
eval_dataset = eval_dataset.batch(batch_size)

In [7]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[64, 20], dtype=Float32, value=
[[ 7.60704160e-01,  4.90134597e-01,  4.61070180e-01 ...  2.53829360e-01,  2.94550419e-01,  2.66070008e-01],
 [ 5.19147158e-01,  9.72505927e-01,  4.82625484e-01 ...  8.15305948e-01,  5.04888296e-02,  9.48516607e-01],
 [ 6.21529102e-01,  9.14839149e-01,  4.85641956e-02 ...  1.86539888e-01,  4.48122025e-01,  5.00619531e-01],
 ...
 [ 6.42607212e-01,  7.54889846e-01,  9.49386358e-02 ...  7.05397606e-01,  3.87017608e-01,  4.22161222e-01],
 [ 7.54454970e-01,  2.57917643e-01,  4.53536391e-01 ...  7.00003386e-01,  1.01274133e-01,  4.73996758e-01],
 [ 3.91571164e-01,  2.09084511e-01,  3.40490103e-01 ...  9.95518088e-01,  5.14354348e-01,  9.36960697e-01]]), 'labels': Tensor(shape=[64], dtype=Int64, value= [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 
 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 
 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1])}


In [8]:
class MLP(nn.Module):
    def __init__(self, num_units_hidden=2000):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(20, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, 2),
            nn.LogSoftmax(dim=-1),
        )

    def forward(self, X):
        return self.seq(X)

In [9]:
lr = 0.002
batch_size = 64
num_epochs = 30

In [10]:
[(n, type(m)) for n, m in MLP().named_modules()]

[('', __main__.MLP),
 ('seq', mindnlp.core.nn.modules.container.Sequential),
 ('seq.0', mindnlp.core.nn.modules.linear.Linear),
 ('seq.1', mindnlp.core.nn.modules.activation.ReLU),
 ('seq.2', mindnlp.core.nn.modules.linear.Linear),
 ('seq.3', mindnlp.core.nn.modules.activation.ReLU),
 ('seq.4', mindnlp.core.nn.modules.linear.Linear),
 ('seq.5', mindnlp.core.nn.modules.activation.LogSoftmax)]

In [11]:
config = peft.LoraConfig(
    r=8,
    target_modules=["seq.0", "seq.2"],
    modules_to_save=["seq.4"],
)

In [12]:
from mindnlp.core import optim
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup

module = MLP()
module_copy = copy.deepcopy(module)  # we keep a copy of the original model for later
model = peft.get_peft_model(module, config)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

print(optimizer)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.002
    maximize: False
    weight_decay: 0
)


In [13]:
model.print_trainable_parameters()

trainable params: 48,160 || all params: 4,096,162 || trainable%: 1.1757347487721432


In [14]:
from tqdm import tqdm

from sys import exit
def forward_fn(**batch):
    outputs = model(batch["input_ids"])
    loss = criterion(outputs, batch["labels"])
    print(loss)
    print(type(loss))
    exit()
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, model.trainable_params())

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer.step(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train(True)
    train_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        # print(batch)
        # exit()
        loss = train_step(**batch)
        train_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with ms._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()

    eval_loss_total = (eval_loss / len(eval_dataset)).item()
    train_loss_total = (train_loss / len(train_dataset)).item()
    print(f"{epoch=:<2}  {train_loss_total=:.4f}  {eval_loss_total=:.4f}")

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:18<?, ?it/s]

0.69333845
<class 'mindspore.common._stub_tensor.StubTensor'>


SystemExit: 

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
# %time train(peft_model, optimizer, criterion, train_dataset, eval_dataset, epochs=num_epochs)

NameError: name 'train_dataloader' is not defined

In [ ]:
for name, param in model.base_model.named_parameters():
    if "lora" not in name:
        continue

    print(f"New parameter {name:<13} | {param.numel():>5} parameters | updated")

In [ ]:
params_before = dict(module_copy.named_parameters())
for name, param in model.base_model.named_parameters():
    if "lora" in name:
        continue

    name_before = (
        name.partition(".")[-1].replace("original_", "").replace("module.", "").replace("modules_to_save.default.", "")
    )
    param_before = params_before[name_before]
    if torch.allclose(param, param_before):
        print(f"Parameter {name_before:<13} | {param.numel():>7} parameters | not updated")
    else:
        print(f"Parameter {name_before:<13} | {param.numel():>7} parameters | updated")